In [1]:
from IPython.display import display, HTML
display(HTML("""
<style>
div.container{width:90% ! important;}
div.cell.code_cell.rendered{width:100%}
div.input_prompt{padding:0px}
div.CodeMirror {font-family:Consolas ; font-size:12pt;}
div.text_cell_render.rendered_html {font-size:12pt;}
div.output {font-size:12pt; font-weight:bold}
div.input {font-family:Consolas ; font-size:12pt;}
div.prompt {min-width:70px;}
div#toc-wrapper {padding-top:120px;}
div.text_cell_render ul li{font-size:12pt;padding:5px;}
table.dataframe{font-size:12px;}
</style>
"""))

<b><font size="6" color="red">ch.14 웹데이터 수집</font></b>
# 1절. BeatifulSoup과 parser
`pip install bs4` 아나콘다를 설치하면 자동 설치되는 7500여개의 패키지에 포함
- 공식 Documentation : https://beautiful-soup-4.readthedocs.io/en/latest/index.html

In [2]:
# 내 local PC의 파일을 url(http://~)처럼 접근
import requests # http 요청처리하는 lib
from requests_file import FileAdapter # file://프로토콜을 다루기 위함

In [3]:
s = requests.Session() # HTTP 요청처리하는 세션 객체
s.mount("file://", FileAdapter())
# file://경로로 들어오면 이 요청은 c:(로컬파일)을 접근
response = s.get('file:///ai/source/01_python/data/ch14_sample.html')
response

<Response [200]>

In [5]:
if response:
    print('잘 접근하였습니다.')
else:
    print('접근을 못했습니다.')

잘 접근하였습니다.


In [6]:
response.status_code
    # 200 : 정상접근
    # 404 : 없는페이지
    # 406 : get. post오류
    # 500 : 파이선 오류(보기 힘듦)

200

In [7]:
response.content # 바이너리 형식의 html내용

b'<!DOCTYPE html>\r\n<html lang="en">\r\n<head>\r\n  <meta charset="UTF-8">\r\n</head>\r\n<body>\r\n  <h1 class="greeting css" id="text">Hello, CSS</h1>\r\n  <h1 class="css">Hi, CSS</h1>\r\n  <div id="subject">subject \xec\x84\xa0\xed\x83\x9d\xec\x9e\x90 \xec\x95\x88\xec\x9d\x98 \xeb\x82\xb4\xec\x9a\xa9</div>\r\n  <p>CSS \xec\x84\xa0\xed\x83\x9d\xec\x9e\x90\xeb\x8a\x94 \xeb\x8b\xa4\xec\x96\x91\xed\x95\x9c \xea\xb3\xb3\xec\x97\x90\xec\x84\x9c \xed\x99\x9c\xec\x9a\xa9\xeb\x90\xa9\xeb\x8b\x88\xeb\x8b\xa4</p>\r\n  <div class="contents">\r\n    \xec\x84\xa0\xed\x83\x9d\xec\x9e\x90\xeb\xa5\xbc \xec\x96\xb4\xeb\x96\xbb\xea\xb2\x8c \xec\x9e\x91\xec\x84\xb1\xed\x95\x98\xeb\x8a\x90\xeb\x83\x90\xec\x97\x90 \xeb\x94\xb0\xeb\x9d\xbc\r\n    <span>\xeb\x8b\xa4\xeb\xa5\xb8<b>\xec\x9a\x94\xec\x86\x8c\xea\xb0\x80 \xeb\xb0\x98\xed\x99\x98</b></span>\xeb\x90\xa9\xeb\x8b\x88\xeb\x8b\xa4\r\n  </div>\r\n  <div>CSS \xec\x84\xa0\xed\x83\x9d\xec\x9e\x90\xeb\x8a\x94 \xeb\x8b\xa4\xec\x96\x91\xed\x95\x9c \xea\xb3\

In [10]:
response.content.decode('utf-8') # 역슬래쉬를 뺴도 싶다면 print()를 이용

'<!DOCTYPE html>\r\n<html lang="en">\r\n<head>\r\n  <meta charset="UTF-8">\r\n</head>\r\n<body>\r\n  <h1 class="greeting css" id="text">Hello, CSS</h1>\r\n  <h1 class="css">Hi, CSS</h1>\r\n  <div id="subject">subject 선택자 안의 내용</div>\r\n  <p>CSS 선택자는 다양한 곳에서 활용됩니다</p>\r\n  <div class="contents">\r\n    선택자를 어떻게 작성하느냐에 따라\r\n    <span>다른<b>요소가 반환</b></span>됩니다\r\n  </div>\r\n  <div>CSS 선택자는 다양한 곳에 <b>활용</b>됩니다</div>\r\n</body>\r\n</html>'

In [11]:
response.text #HTML파일의 텍스트 내용

'<!DOCTYPE html>\r\n<html lang="en">\r\n<head>\r\n  <meta charset="UTF-8">\r\n</head>\r\n<body>\r\n  <h1 class="greeting css" id="text">Hello, CSS</h1>\r\n  <h1 class="css">Hi, CSS</h1>\r\n  <div id="subject">subject 선택자 안의 내용</div>\r\n  <p>CSS 선택자는 다양한 곳에서 활용됩니다</p>\r\n  <div class="contents">\r\n    선택자를 어떻게 작성하느냐에 따라\r\n    <span>다른<b>요소가 반환</b></span>됩니다\r\n  </div>\r\n  <div>CSS 선택자는 다양한 곳에 <b>활용</b>됩니다</div>\r\n</body>\r\n</html>'

In [12]:
# html 파싱
from bs4 import BeautifulSoup
soup=BeautifulSoup(response.content, #response.text
                  'html.parser')
# soup

<!DOCTYPE html>

<html lang="en">
<head>
<meta charset="utf-8"/>
</head>
<body>
<h1 class="greeting css" id="text">Hello, CSS</h1>
<h1 class="css">Hi, CSS</h1>
<div id="subject">subject 선택자 안의 내용</div>
<p>CSS 선택자는 다양한 곳에서 활용됩니다</p>
<div class="contents">
    선택자를 어떻게 작성하느냐에 따라
    <span>다른<b>요소가 반환</b></span>됩니다
  </div>
<div>CSS 선택자는 다양한 곳에 <b>활용</b>됩니다</div>
</body>
</html>

In [19]:
# soup.select_one('선택자') : 해당선택자 처음 하나만
el = soup.select_one('h1') # 처음 나오는 h1태그 하나만
print('el :',el)
print('el.text :', el.text)
print('el.string :', el.string)
print('el의 속성들:', el.attrs)
print('el의 id속성:', el.attrs['id']) # el.attrs는 딕셔너리
print('el의 id속성:', el.attrs.get('id'))
print('el의 href속성:', el.attrs.get('href'))
print('el의 태그이름:', el.name)

el : <h1 class="greeting css" id="text">Hello, CSS</h1>
el.text : Hello, CSS
el.string : Hello, CSS
el의 속성들: {'class': ['greeting', 'css'], 'id': 'text'}
el의 id속성: text
el의 id속성: text
el의 href속성: None
el의 태그이름: h1


In [25]:
# soup.select('선택자') : 해당 선택자 모두
els=soup.select('h1') # h1태그를 리스트로 없는 태그는 빈 list
print('els :', els)
print("els의 text들 :", [el.text for el in els])
print("els의 string들 :", [el.string for el in els])
print("els의 속성들 :", [el.attrs for el in els])
print("els의 class 속성들 :", [el.attrs.get('class') for el in els])

els : [<h1 class="greeting css" id="text">Hello, CSS</h1>, <h1 class="css">Hi, CSS</h1>]
els의 text들 : ['Hello, CSS', 'Hi, CSS']
els의 string들 : ['Hello, CSS', 'Hi, CSS']
els의 속성들 : [{'class': ['greeting', 'css'], 'id': 'text'}, {'class': ['css']}]
els의 class 속성들 : [['greeting', 'css'], ['css']]


In [31]:
# soup.find(태그, 속성) : soup,select_one('선택자')와 유사
print('select_one: ',soup.select_one('h1.css'))
print('find : ', soup.find('h1',{'class':'css'}))# soup.select_one('h1.css')
print('find : ', soup.find('h1',class_='css'))
print()
print('select_one:',soup.select_one('h1#text'))
print('find :', soup.find('h1',{'id':'text'}))

select_one:  <h1 class="greeting css" id="text">Hello, CSS</h1>
find :  <h1 class="greeting css" id="text">Hello, CSS</h1>
find :  <h1 class="greeting css" id="text">Hello, CSS</h1>

select_one: <h1 class="greeting css" id="text">Hello, CSS</h1>
find : <h1 class="greeting css" id="text">Hello, CSS</h1>


In [36]:
# soup.find_all(태그, 속성) : soup.select('선택자')와 유사
print('select :',soup.select('h1.css'))
print('find_all:', soup.find_all('h1',class_='css'))
print('find_all:', soup.find_all('h1',{'class':'css'}))

select : [<h1 class="greeting css" id="text">Hello, CSS</h1>, <h1 class="css">Hi, CSS</h1>]
find_all: [<h1 class="greeting css" id="text">Hello, CSS</h1>, <h1 class="css">Hi, CSS</h1>]
find_all: [<h1 class="greeting css" id="text">Hello, CSS</h1>, <h1 class="css">Hi, CSS</h1>]


In [40]:
print('select :',soup.select('h1.css, span'))
print('find_all :', soup.find_all(['h1','span'],class_='css') +\
                    soup.find_all('span'))

select : [<h1 class="greeting css" id="text">Hello, CSS</h1>, <h1 class="css">Hi, CSS</h1>, <span>다른<b>요소가 반환</b></span>]
find_all : [<h1 class="greeting css" id="text">Hello, CSS</h1>, <h1 class="css">Hi, CSS</h1>, <span>다른<b>요소가 반환</b></span>]


In [42]:
# 없는 엘리먼트 찾기
print('select(빈 list):', soup.select('a.css'))
print('select_one(None) :',soup.select_one('a.css'))
print('find_all(빈 list):', soup.find_all('a',{'class':'css'}))
print('find(none):', soup.find('a',{'class','css'}))

select(빈 list): []
select_one(None) : None
find_all(빈 list): []
find(none): None


# 2절. 정적 웹 데이터 수집(정적 웹크롤링)
## 2.1 BeartifulSoup을 활용한 html 웹 데이터 수집
### 1) 환율정보 가져오기(네이버증권->시장지표)
- https://finance.naver.com/marketindex/

In [ ]:
# 크롤링 허용범위는 사이트마다 ~/robots.txt 에서 확인 ex)https://finance.naver.com/robots.txt
    # Allow: / - 사이트의 모든 경로(/)에 대한 크롤링 허용

In [48]:
# 방법1 url에 한글 포함되어도 관계없음
import requests
from bs4 import BeautifulSoup
url = 'https://finance.naver.com/marketindex/'
response = requests.get(url)
# response.status_code
# response.text / response.content.decode('cp-949')
soup=BeautifulSoup(response.text,'html.parser')

In [53]:
# 방법2
from urllib.request import urlopen
url = 'https://finance.naver.com/marketindex/'
response = urlopen(url)
response.status
# response.read() / response.read().decode('cp949')
soup = BeautifulSoup(response,'html.parser')

In [60]:
# div.head_info > span.value (find함수)
prices = []
headinfos = soup.find_all('div',class_='head_info')
for headinfo in headinfos:
    price = headinfo.find('span', class_='value')
#     print(price.text.replace(',',''))
    prices.append((''.join(price.text.split(','))))
print(prices)

['1454.50', '945.19', '1681.33', '204.20', '153.4200', '1.1564', '1.3161', '99.4700', '59.75', '1698.27', '4009.8', '189125.23']


In [65]:
# div.head_info > span.value (select함수)
prices = soup.select('div.head_info > span.value')
prices
print([float(p.text.replace(",",'')) for p in prices])

[1454.5, 945.19, 1681.33, 204.2, 153.42, 1.1564, 1.3161, 99.47, 59.75, 1698.27, 4009.8, 189125.23]


In [74]:
titles = soup.select('h3.h_lst > span.blind')
titles
print([t.text for t in titles])

['미국 USD', '일본 JPY(100엔)', '유럽연합 EUR', '중국 CNY', '달러/일본 엔', '유로/달러', '영국 파운드/달러', '달러인덱스', 'WTI', '휘발유', '국제 금', '국내 금']


In [79]:
units = soup.select('div.head_info > span > span.blind')
units = [u.text for u in units]
units.insert(7, "") #7번째에  "" 추가
units

['원', '원', '원', '원', '엔', '달러', '달러', '', '달러', '원', '달러', '원']

In [81]:
statuses = soup.select('div.head_info > span.blind')
statuses = [s.text for s in statuses]
statuses

['하락', '하락', '하락', '하락', '상승', '하락', '상승', '하락', '상승', '상승', '상승', '상승']

In [82]:
len(titles), len(prices), len(units), len(statuses)

(12, 12, 12, 12)

In [87]:
for idx in range(len(titles)):
    print(f"{titles[idx].text} : {prices[idx].text}{units[idx]}-{status[idx].text}")

미국 USD : 1,454.50원-하락
일본 JPY(100엔) : 945.19원-하락
유럽연합 EUR : 1,681.33원-하락
중국 CNY : 204.20원-하락
달러/일본 엔 : 153.4200엔-상승
유로/달러 : 1.1564달러-하락
영국 파운드/달러 : 1.3161달러-상승
달러인덱스 : 99.4700-하락
WTI : 59.75달러-상승
휘발유 : 1698.27원-상승
국제 금 : 4009.8달러-상승
국내 금 : 189125.23원-상승


In [95]:
for title, price, unit, status in zip(titles, prices, units, statuses):
    print(f"{title.text} : {price.text}{unit}-{status}")

미국 USD : 1,454.50원-하락
일본 JPY(100엔) : 945.19원-하락
유럽연합 EUR : 1,681.33원-하락
중국 CNY : 204.20원-하락
달러/일본 엔 : 153.4200엔-상승
유로/달러 : 1.1564달러-하락
영국 파운드/달러 : 1.3161달러-상승
달러인덱스 : 99.4700-하락
WTI : 59.75달러-상승
휘발유 : 1698.27원-상승
국제 금 : 4009.8달러-상승
국내 금 : 189125.23원-상승


### 2) 이번 주 로또 번호 출력
- https://dhlottery.co.kr/gameResult.do?method=byWin
```
    1197회 (2025년 11월 08일 추첨)
    당첨번호 [1,5,7,26,28,43]
    보너스 30
```

In [96]:
import requests
from bs4 import BeautifulSoup
url = 'https://dhlottery.co.kr/gameResult.do?method=byWin'
response = requests.get(url)
soup=BeautifulSoup(response.text,'html.parser')

In [105]:
count = soup.select_one('h4 > strong')
count

<strong>1197회</strong>

In [104]:
date = soup.select_one('p.desc')
date

<p class="desc">(2025년 11월 08일 추첨)</p>

In [148]:
balls = soup.select('div.num.win > p > span.ball_645')
balls = [int(b.text) for b in  balls]
balls

[1, 5, 7, 26, 28, 43]

In [151]:
bball = soup.select_one('div.num.bonus> p > span.ball_645') #num bonus인 경우 class가 num이면서 bonus이기에 num.bonus를 사용하면 된다.
print(bball)

<span class="ball_645 lrg ball3">30</span>


In [152]:
print(count.text + " " + date.text)
print(f"당첨번호 {balls}") 
print("보너스 " + bball.text)

1197회 (2025년 11월 08일 추첨)
당첨번호 [1, 5, 7, 26, 28, 43]
보너스 30


### 3) 다음 검색 리스트
```
no          title                                     link
0 "한풀 꺾인 비트코인, 상승세 곧 회복"…근거는?    https://v.daum.net/v/20251110094711892
```

In [186]:
# 방법1
word ="비트코인"
url = 'https://search.daum.net/search?w=news&nil_search=btn&DA=NTB&enc=utf8&cluster=y&cluster_page=1&q=' + word
print(url)
response = requests.get(url)
soup=BeautifulSoup(response.text, 'html.parser')

https://search.daum.net/search?w=news&nil_search=btn&DA=NTB&enc=utf8&cluster=y&cluster_page=1&q=비트코인


In [185]:
# # 방법2 : url에 한글 불가(한글을 ascii로 변경해야 가능
# from urllib.request import urlopen
# from urllib.parse import quote
# word = quote('비트코인') # url에 한글이 포함된 경우 한글을 quote()로 변환
# url = 'https://search.daum.net/search?w=news&nil_search=btn&DA=NTB&enc=utf8&cluster=y&cluster_page=1&q=' + word
# print(url)
# response = urlopen(url)
# soup = BeautifulSoup(response,'html.parser')

In [187]:
items_find_list = [] # 검색한 결과를 담을 리스트(딕셔너리 list)
items_el=soup.select('div.item-title > strong.tit-g > a')
item_el[0]
for idx, item in enumerate(items_el):
   # print(idx, item.text, item.attrs.get('href'))
    items_find_list.append({'no':idx, 'title' : item.text, 'link' : item.attrs.get('href')})

pd.DataFrame(items_find_list)

,no,title,link
0,0,"""한풀 꺾인 비트코인, 상승세 곧 회복""…근거는?",http://v.daum.net/v/20251110094711892
1,1,비트코인 1억5500만원대 소폭 상승…트럼프 발언탓?,http://v.daum.net/v/20251110092814031
2,2,"""비트코인 21억 간다더니""...갑자기 말 바꿨다",http://v.daum.net/v/20251109105545139
3,3,"트럼프 돈 푼다...비트코인, 관세 배당금에 반등 [매일코인]",http://v.daum.net/v/20251110112101913
4,4,"비트코인, 트럼프 ‘2000달러 지급’ 발언에 10만4000달러선 [크립토브리핑]",http://v.daum.net/v/20251110103033108
5,5,"“비트코인 약세, 닷컴버블과 판박이”…큰손·장투족 매일 던진다",http://v.daum.net/v/20251110094507791
6,6,비트코인 '와르르' 내려오는데 '의외'의 전망,http://v.daum.net/v/20251107210115339
7,7,"[가상자산 나침반] ‘겹악재’ 비트코인, 반전 포인트 나올까",http://v.daum.net/v/20251109112746909
8,8,"美 셧다운 종료 기대…비트코인, 10만 5000달러 회복[코인브리핑]",http://v.daum.net/v/20251110111435519
9,9,"비트플래닛, 비트코인 200개 매입 완료…자금 조달부터 커스터디까지 끝내",http://v.daum.net/v/20251110081500340


In [190]:
items_find_list = [] # 검색한 결과를 담을 리스트(딕셔너리 list)
items_el=soup.select('div.item-title > strong.tit-g > a')
item_el[0]
for idx, item in enumerate(items_el):
   # print(idx, item.text, item.attrs.get('href'))
    items_find_list.append([idx, item.text, item.attrs['href']])

pd.DataFrame(items_find_list, columns=['번호', '제목', '링크'])

,번호,제목,링크
0,0,"""한풀 꺾인 비트코인, 상승세 곧 회복""…근거는?",http://v.daum.net/v/20251110094711892
1,1,비트코인 1억5500만원대 소폭 상승…트럼프 발언탓?,http://v.daum.net/v/20251110092814031
2,2,"""비트코인 21억 간다더니""...갑자기 말 바꿨다",http://v.daum.net/v/20251109105545139
3,3,"트럼프 돈 푼다...비트코인, 관세 배당금에 반등 [매일코인]",http://v.daum.net/v/20251110112101913
4,4,"비트코인, 트럼프 ‘2000달러 지급’ 발언에 10만4000달러선 [크립토브리핑]",http://v.daum.net/v/20251110103033108
5,5,"“비트코인 약세, 닷컴버블과 판박이”…큰손·장투족 매일 던진다",http://v.daum.net/v/20251110094507791
6,6,비트코인 '와르르' 내려오는데 '의외'의 전망,http://v.daum.net/v/20251107210115339
7,7,"[가상자산 나침반] ‘겹악재’ 비트코인, 반전 포인트 나올까",http://v.daum.net/v/20251109112746909
8,8,"美 셧다운 종료 기대…비트코인, 10만 5000달러 회복[코인브리핑]",http://v.daum.net/v/20251110111435519
9,9,"비트플래닛, 비트코인 200개 매입 완료…자금 조달부터 커스터디까지 끝내",http://v.daum.net/v/20251110081500340


In [198]:
items_find_list = []
# div.item-title > strong.tit-g > a
item_tltles = soup.find_all('div', class_='item-title')
for idx, item in enumerate(item_tltles):
    a = item.find('a')
    #print(idx, a.text, a.attrs['href'])
    items_find_list.append([idx, a.text, a.attrs['href']])
pd.DataFrame(items_find_list, columns=['번호', '제목', '링크'])

,번호,제목,링크
0,0,"""한풀 꺾인 비트코인, 상승세 곧 회복""…근거는?",http://v.daum.net/v/20251110094711892
1,1,비트코인 1억5500만원대 소폭 상승…트럼프 발언탓?,http://v.daum.net/v/20251110092814031
2,2,"""비트코인 21억 간다더니""...갑자기 말 바꿨다",http://v.daum.net/v/20251109105545139
3,3,"트럼프 돈 푼다...비트코인, 관세 배당금에 반등 [매일코인]",http://v.daum.net/v/20251110112101913
4,4,"비트코인, 트럼프 ‘2000달러 지급’ 발언에 10만4000달러선 [크립토브리핑]",http://v.daum.net/v/20251110103033108
5,5,"“비트코인 약세, 닷컴버블과 판박이”…큰손·장투족 매일 던진다",http://v.daum.net/v/20251110094507791
6,6,비트코인 '와르르' 내려오는데 '의외'의 전망,http://v.daum.net/v/20251107210115339
7,7,"[가상자산 나침반] ‘겹악재’ 비트코인, 반전 포인트 나올까",http://v.daum.net/v/20251109112746909
8,8,"美 셧다운 종료 기대…비트코인, 10만 5000달러 회복[코인브리핑]",http://v.daum.net/v/20251110111435519
9,9,"비트플래닛, 비트코인 200개 매입 완료…자금 조달부터 커스터디까지 끝내",http://v.daum.net/v/20251110081500340


In [254]:
# 다음 뉴스 검색(원하는 키워드를 원하는 페이지를 가져오기)
import requests
from bs4 import BeautifulSoup
import pandas as pd
def collect_news_list(keyword, page):
    'keyword로 page애 다음 뉴스 검색한 결과를 출력 -> list를 return'
    #url = f'https://search.daum.net/search?w=news&nil_search=btn&DA=PGD&enc=utf8&cluster=y&cluster_page=1&q={keyword}&p={page}'
    #response = requests.get(url)
    url = 'https://search.daum.net/search?w=news&nil_search=btn&DA=PGD&enc=utf8'
    params = {'q' : keyword, "p" : page}
    response = requests.get(url, params = params)
    soup = BeautifulSoup(response.text,"html.parser")
    items_find_list = [] # 검색한 결과를 받는 리스트
    item_titles = soup.select('div.item-title > strong.tit-g > a')
    for idx, item in enumerate(item_titles):
        items_find_list.append([(page-1)*10+idx, item.text, item.attrs['href']])
    return items_find_list

In [257]:
collect_news_list('청바지',2)

[]

In [262]:
import time
keywords=['청바지', '동대문']
result0 = [] # 청바지 1~3페이지까지 검색한 결과를 담을 list
result1 = [] # 동대문 1~3페이지까지 검색한 결과를 담을 list
pages = 3
for i, keyword in enumerate(keywords):
    print(f'==={i+1}번째 검색어 {keyword} 검색 결과를 수집 중입니다.===')
    for page in range(1, pages+1):
        if i == 0:
            result0.extend(collect_news_list(keyword, page))
        elif i == 1:
            result1.extend(collect_news_list(keyword, page))
        time.sleep(0.5)
        

===1번째 검색어 청바지 검색 결과를 수집 중입니다.===
===2번째 검색어 동대문 검색 결과를 수집 중입니다.===


In [263]:
result0_df=pd.DataFrame(result0, columns=['no','title','link'])
result1_df=pd.DataFrame(result1, columns=['no','title','link'])
result0_df

,no,title,link
0,0,"'44kg' 송가인, 빵 이렇게 먹는데 살 더 빠졌나봐…청바지가 헐렁[SCin스타]",http://v.daum.net/v/20251109085414195
1,1,"'44kg' 송가인, 반려견과 행복 일상…'늘씬' 청바지 패션 '눈길' [★해시태그]",http://v.daum.net/v/20251109175251684
2,2,"'청바지 광고 논란' 시드니 스위니, 처음으로 입장 밝혔다",http://v.daum.net/v/20251105085146790
3,3,"‘다이어트 성공’ 송가인, 늘씬한 청바지 핏",http://v.daum.net/v/20251110084450093
4,4,맨날 청바지만 입는다면 클릭하세요,http://v.daum.net/v/20251105051149510
5,5,"‘45kg’ 윤은혜, 완벽한 화이트 티+청바지로 뽐낸 완벽 비율..’늘씬한 자태’",http://v.daum.net/v/20251107202047755
6,6,"김윤지, 출산 후 16kg 감량→47kg 유지…무결점 청바지 핏 '감탄' (ns윤지)",http://v.daum.net/v/20251103120847620
7,7,"'200억 재산설' 송가인, 이렇게 말랐었나…청바지핏 드러내며 미모자랑",http://v.daum.net/v/20251109083254977
8,8,"'실컷 먹어도 47kg 유지' 김윤지 ""흰티에 청바지만 입어""..출산 후 놀라운 몸매",http://v.daum.net/v/20251103091538134
9,9,"청하, 잘못 본 줄 알았다? 청바지 위에 청바지 같은 '파격 스타일링' [MD픽]",http://v.daum.net/v/20251030033511171


In [264]:
result0_df.to_csv(f'data/ch14_{keywords[0]}.csv', index=False)
result1_df.to_csv(f'data/ch14_{keywords[1]}.csv', index=False)

### 4) User-Agent를 추가하여 크롤링
- 자동으로 브라우저를 통해 요청하는 것처럼 보이기

In [261]:
# 방법 2 :
from urllib.request import urlopen,Request
from urllib.parse import quote
word = quote('비트코인')
url = 'https://search.daum.net/search?w=news&nil_search=btn&DA=PGD&enc=utf8&q='+word
print(url)
# headers={'user-Agent':
#          'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/142.0.0.0 Safari/537.36'}
# request=Request(url, headers=headers)
request=Request(url)
request.add_header{'user-Agent':
                   'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/142.0.0.0 Safari/537.36'}
response=urlopen(request)
soup = BeautifulSoup(response,'html.parser')
# soup

https://search.daum.net/search?w=news&nil_search=btn&DA=PGD&enc=utf8&q=%EB%B9%84%ED%8A%B8%EC%BD%94%EC%9D%B8


<!DOCTYPE html>

<html lang="ko" xmlns="http://www.w3.org/1999/xhtml">
<head profile="http://a9.com/-/spec/opensearch/1.1/">
<meta content="text/html;charset=utf-8" http-equiv="content-Type"/>
<meta content="IE=edge" http-equiv="X-UA-Compatible"/>
<meta content="off" name="autocomplete"/>
<meta content="always" name="referrer"/>
<meta content="telephone=no,address=no,email=no" name="format-detection">
<meta content="비트코인 – Daum 검색" property="og:title"/>
<meta content="https://search.daum.net/search?w=news&amp;nil_search=btn&amp;DA=PGD&amp;enc=utf8&amp;q=%EB%B9%84%ED%8A%B8%EC%BD%94%EC%9D%B8" property="og:url"/>
<meta content="Daum 검색에서 비트코인에 대한 최신정보를 찾아보세요." property="og:description"/>
<meta content="https://search1.daumcdn.net/search/statics/common/img/20250107/daum_og.png" property="og:image"/>
<meta content="다음검색" property="og:site_name"/>
<link href="https://t1.daumcdn.net/top/favicon/20241223/daum.ico" rel="shortcut icon"/>
<link href="//sl.search.daum.net/" rel="preconnect"/>
<lin

### 5)멜론차트
- https://www.melon.com/chart/index.htm
   - https://www.melon.com/robots.txt 에서 User-Agent에 봇이 지정

In [269]:
# 방법1
import requests
url = 'https://www.melon.com/chart/index.htm'
melonpage = requests.get(url)
melonpage
# soup = BeautifulSoup(melonpage.text, 'html.parser')
# soup

<Response [406]>

In [274]:
# 방법2 : user-agent추가
from urllib.request import urlopen, Request
url = 'https://www.melon.com/chart/index.htm'
# melonpage = urlopen(url) # 406에러
header = {'User-Agent':
          'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/142.0.0.0 Safari/537.36'}
request = Request(url, headers=headers)
melonpage = urlopen(request)
soup = BeautifulSoup(melonpage,'html.parser')
# soup

<!DOCTYPE html>

<html lang="ko">
<head>
<meta content="text/html; charset=utf-8" http-equiv="Content-Type"/>
<meta content="IE=edge,chrome=1" http-equiv="X-UA-Compatible"/>
<title>멜론차트&gt;TOP100&gt;멜론</title>
<meta content="음악서비스, 멜론차트, 멜론TOP100, 최신음악, 인기가요, 뮤직비디오, 앨범, 플레이어, 스트리밍, 다운로드, 아티스트플러스, 아티스트채널" name="keywords"/>
<meta content="No.1 뮤직플랫폼 멜론! 최신 트렌드부터 나를 아는 똑똑한 음악추천까지!" name="description"/>
<meta content="ee85ff6db1fa8f2226bcb671ecb2bcdbcffb6f8b" name="naver-site-verification"/>
<meta content="q4tbTQhmxa4La3OdNLsNOCxrJ_WV6lUlBFrFW4-HqQc" name="google-site-verification"/>
<meta content="4022717807957185" property="fb:app_id"/>
<meta content="Melon" property="og:title"/>
<meta content="https://cdnimg.melon.co.kr/resource/image/web/common/logo_melon142x99.png" property="og:image"/>
<meta content="음악이 필요한 순간, 멜론" property="og:description"/>
<meta content="http://www.melon.com/chart/index.htm" property="og:url"/>
<meta content="website" property="og:type"/>
<meta content="멜론" prope

In [293]:
# 방법1 : user-agent 추가
url = 'https://www.melon.com/chart/index.htm'
header = {'User-Agent':
          'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/142.0.0.0 Safari/537.36'}
melonpage = requests.get(url, headers=headers)
soup = BeautifulSoup(melonpage.text, # melon.content
                     'html.parser')

ranks = soup.select('td div.wrap.t_center > span.rank')
titles = soup.select('div.ellipsis.rank01 > span > a') #[title.text.strip() for title in titles]
singers = soup.select('div.ellipsis.rank02') #[singer.text.strip()[:20] for singer in singers]
# 1위 | 노래재목, 가수명
for rank, title, singer in zip(ranks, titles, singers):
    print(f'{rank.text} 위 | {title.text.strip()} - {singer.text.strip()[:20]}')

1 위 | Blue Valentine - NMIXXNMIXX
2 위 | 타임캡슐 - 다비치다비치
3 위 | Golden - HUNTR/X, EJAE, AUDRE
4 위 | Drowning - WOODZWOODZ
5 위 | Good Goodbye - 화사 (HWASA)화사 (HWASA)
6 위 | SPAGHETTI (feat. j-hope of BTS) - LE SSERAFIM (르세라핌), 
7 위 | 달리 표현할 수 없어요 - 로이킴로이킴
8 위 | 뛰어(JUMP) - BLACKPINKBLACKPINK
9 위 | Soda Pop - KPop Demon Hunters C
10 위 | 어제보다 슬픈 오늘 - 우디 (Woody)우디 (Woody)
11 위 | FAMOUS - ALLDAY PROJECTALLDAY
12 위 | 시작의 아이 ❍ - 박다혜, 마크툽 (MAKTUB)박다혜
13 위 | IRIS OUT - Kenshi YonezuKenshi 
14 위 | XOXZ - IVE (아이브)IVE (아이브)
15 위 | 모르시나요(PROD.로코베리) - 조째즈조째즈
16 위 | Rich Man - aespaaespa
17 위 | Hollywood Action - BOYNEXTDOORBOYNEXTDO
18 위 | 한번 더 이별 - 이창섭이창섭
19 위 | 내게 사랑이 뭐냐고 물어본다면 - 로이킴로이킴
20 위 | 너에게 닿기를 - 10CM10CM
21 위 | HOME SWEET HOME (feat. 태양, 대성) - G-DRAGONG-DRAGON
22 위 | 운명 (2025) - 먼데이 키즈, 이이경먼데이 키즈, 이
23 위 | body - 다영 (DAYOUNG)다영 (DAYO
24 위 | toxic till the end - 로제 (ROSÉ)로제 (ROSÉ)
25 위 | Whiplash - aespaaespa
26 위 | 시작의 아이 - 마크툽 (MAKTUB)마크툽 (MAK
27 위 | 천상연 - 이창섭이창섭
28 위 | 나는 반딧불 - 황가람황가람
29 위 | 어

### 6) 네이버지식인 검색(openApi 사용 X)
- 특정 keyword를 특정 페이지수만큼

In [7]:
# 방법1
from requests import get
from bs4 import BeautifulSoup
keyword='쳇지피티'
# 파라미터 x
url = f'https://kin.naver.com/search/list.naver?query={keyword}'
response=get(url)
# 파라미터 사용
# url = f'https://kin.naver.com/search/list.naver'
# params={'query' : keyword}
# response=get(url, params=params)
print(response.status_code)
soup=BeautifulSoup(response.text,'html.parser')
soup

200



<!DOCTYPE html>

<html lang="ko">
<head>
<meta contents="always" name="referrer"/>
<meta content="IE=edge" http-equiv="X-UA-Compatible"/>
<meta content="쳇지피티의 지식iN Q&amp;A 검색결과입니다. 궁금증을 해결하지 못했다면 지식iN '질문하기'를 해보세요." name="description">
<meta content="width=1024" name="viewport"/>
<meta content="none" name="msapplication-config">
<link href="https://ssl.pstatic.net/static.kin/static/pc/20251029142034/css/min/common.css" rel="stylesheet" type="text/css"/>
<link href="https://ssl.pstatic.net/static.kin/static/pc/20251029142034/css/min/components.css" rel="stylesheet" type="text/css"/>
<link href="https://ssl.pstatic.net/static.kin/static/pc/20251029142034/css/min/other.css" rel="stylesheet" type="text/css"/>
<script>
	
	var pcDomain = "kin.naver.com";
	var pcDomainWithProtocol = "https://kin.naver.com";
	var pcKinServiceProtocol = "https";

	var mobileDomain = "m.kin.naver.com";
	var mobileDomainWithProtocol = "https://m.kin.naver.com";
	var mobileKinServiceProtocol = "https";

	var uplo

In [12]:
# 방법2
from urllib.request import urlopen
from urllib.parse import quote
keyword=quote('쳇지피티')
url = f'https://kin.naver.com/search/list.naver?query={keyword}'
print(url)
response = urlopen(url)
print(response.status)
soup =BeautifulSoup(response,'html.parser')
# soup

https://kin.naver.com/search/list.naver?query=%EC%B3%87%EC%A7%80%ED%94%BC%ED%8B%B0
200


In [24]:
# 페이징 포함
from urllib.request import urlopen
from urllib.parse import quote
from bs4 import BeautifulSoup
import pandas as pd
keyword = quote('챗지피티')
pages = 3
items_list=[] # 크롤링한 데이터를 담을 list(title,link)
for page in range(1,pages+1):
    url=f'https://kin.naver.com/search/list.naver?query={keyword}&page={page}'
    response = urlopen(url)
    #print(response.status)
    soup = BeautifulSoup(response,'html.parser')
    items = soup.select('dt > a')
    for item in items:
        title = item.text
        link = item.attrs.get('href')
        items_list.append({
            'title':title,
            'link':link
        })
df = pd.DataFrame(items_list)
print(df.shape)
df

(30, 2)


,title,link
0,챗지피티 관련 질문,https://kin.naver.com/qna/detail.naver?d1id=1&...
1,챗지피티 자동 결제 안 됨,https://kin.naver.com/qna/detail.naver?d1id=1&...
2,챗지피티 쉐어,https://kin.naver.com/qna/detail.naver?d1id=1&...
3,챗지피티 전송 오류,https://kin.naver.com/qna/detail.naver?d1id=1&...
4,Ai 챗지피티 답변,https://kin.naver.com/qna/detail.naver?d1id=1&...
5,챗지피티 오류 고치는 방법 있나요?,https://kin.naver.com/qna/detail.naver?d1id=13...
6,요즘 쓰는 챗지피티,https://kin.naver.com/qna/detail.naver?d1id=1&...
7,챗지피티 영어 문법,https://kin.naver.com/qna/detail.naver?d1id=11...
8,챗지피티 타로,https://kin.naver.com/qna/detail.naver?d1id=3&...
9,챗지피티 자기소개서,https://kin.naver.com/qna/detail.naver?d1id=4&...


## 2.2 openAPI사용 :json 웹데이터 수집
### 1) 네이버 지식인으로 검색 (openAPI 사용 O)
- 네이버 개발자센터에서 애플래케이션등록(이름/검색/web설정http://localhost)
- .env파일에 CLIENT_ID / CLIENT_SECRET 환경변수 저장
- 환경변수를 읽기 위해서  `pip install dotenv`
- 특정 keyword를 지식검색(데이터 수 30개)

In [25]:
%pip install dotenv

Note: you may need to restart the kernel to use updated packages.


In [27]:
# 환경변수 읽어오기
from dotenv import load_dotenv
import os
load_dotenv(
        #dotenv_path='.env'
            )
# 현 소스와 같은 폴더 내의 .env흫 메모리에 load
# print(os.getenv('CLIENT_ID'))
# print(os.getenv('CLIENT_SECRET'))

True

In [42]:
# 방법 2
import os
import urllib.request
import json
client_id = os.getenv('CLIENT_ID')
client_secret = os.getenv('CLIENT_SECRET')
encText = urllib.parse.quote("챗지피티")
url = "https://openapi.naver.com/v1/search/kin.json?query=" + encText # JSON 결과
# url = "https://openapi.naver.com/v1/search/blog.xml?query=" + encText # XML 결과

# request = urllib.request.Request(url)
# request.add_header("X-Naver-Client-Id",client_id)
# request.add_header("X-Naver-Client-Secret",client_secret)
headers = {
    'X-Naver-Client-Id':client_id,
    'X-Naver-Client-Secret':client_secret
}
request=urllib.request.Request(url,headers=headers)

response = urllib.request.urlopen(request)

rescode = response.status
if(rescode==200):
    response_body = response.read()
    print(type(response_body.decode('utf-8')))
    print(response_body.decode('utf-8')[:50])
else:
    print("Error Code:" + rescode)

<class 'str'>
{
	"lastBuildDate":"Tue, 11 Nov 2025 11:48:17 +090


In [54]:
#방법1
import requests
import pandas as pd
import json # response 택스트를 json 스타일의 딕셔너리로
client_id = os.getenv('CLIENT_ID')
client_secret = os.getenv('CLIENT_SECRET')
query ='챗지피티'
headers = {
    'X-Naver-Client-Id':client_id,
    'X-Naver-Client-Secret':client_secret
}
# url = f"https://openapi.naver.com/v1/search/kin.json?query={query}&display=30"
# response = requests.get(url, headers=headers)
url="https://openapi.naver.com/v1/search/kin"
params={'query':query,'display':30}
response = requests.get(url,params=params, headers=headers)
# print(response.text[:500])

# items = json.loads(response.text)['items']
items = response.json()['items']
items_list = []
for item in items:
    title = item['title'].replace('<b>','').replace('</b>','')
    link = item['link']
    description = item.get('description').replace('<b>','').replace('</b>','')
    items_list.append([title, link, description])
pd.DataFrame(items_list,columns=['title','link','description'])

,title,link,description
0,챗지피티 관련 질문,https://kin.naver.com/qna/detail.naver?d1id=1&...,"챗지피티 계정 삭제시 챗지피티에서 활동했던 기록(대화, 사진 등)들 다 서버에서 삭..."
1,챗지피티 자동 결제 안 됨,https://kin.naver.com/qna/detail.naver?d1id=1&...,7월 7일에 챗지피티 프로 결제해서 한달 열심히 썼습니다. 이번 달에도 7일쯤......
2,챗지피티 쉐어,https://kin.naver.com/qna/detail.naver?d1id=1&...,... 에서 챗지피티 쉐어를 하려고 하는데 챗지피티는 계정공유가 어떤식으로 진행되는...
3,챗지피티 전송 오류,https://kin.naver.com/qna/detail.naver?d1id=1&...,챗지피티 메세지 전송하면 자꾸 저렇게 떠요 ㅠㅠ 이 오류는 보통 서버 과부하나 시스...
4,Ai 챗지피티 답변,https://kin.naver.com/qna/detail.naver?d1id=1&...,... 사람이 하면 그래도 뿌듯함이라도 느끼는데 굳이 챗지피티 써가면서 감정없는 답...
5,챗지피티 오류 고치는 방법 있나요?,https://kin.naver.com/qna/detail.naver?d1id=13...,제가 챗지피티 유저인데 자꾸 첨부 파일이 있습니다 뭐시기 이런게 떠요.... 로그인...
6,요즘 쓰는 챗지피티,https://kin.naver.com/qna/detail.naver?d1id=1&...,... 챗지피티 많이 쓰던데 저는 갤럭시지챗지피티줄 알았는데 아이폰도 쓰길래 궁금하...
7,챗지피티 영어 문법,https://kin.naver.com/qna/detail.naver?d1id=11...,챗지피티로 영어 문법 오류 유무 물어봤는데 지피티 문법도 믿을만한가요?? 둘 다 절...
8,챗지피티 타로,https://kin.naver.com/qna/detail.naver?d1id=3&...,"챗지피티 타로 믿을만한가요?ㅜㅜ 챗지피티 타로, 그러니까 인공지능이... 요즘엔 온..."
9,챗지피티 자기소개서,https://kin.naver.com/qna/detail.naver?d1id=4&...,챗지피티에게 저의 경험과 경력 등을 제시하고 챗지피티가 써준 그대로 옮겨적으면 문제...


### quiz) 네이버 oprnAPI를 이용해서 청바지 이미니 100건의 데이터를 ch14_청바지.csv출력
```
    제목(title), 링크, 썸네일, sizeheight, sizewidth
```

In [67]:
from urllib.request import urlopen,Request
from urllib.parse import quote
import pandas as pd
import os
from dotenv import load_dotenv

client_id = os.getenv('CLIENT_ID')
client_secret = os.getenv('CLIENT_SECRET')

query =quote('청바지')
headers = {
    'X-Naver-Client-Id':client_id,
    'X-Naver-Client-Secret':client_secret
}

url=f'https://openapi.naver.com/v1/search/image?query={query}&display=100'
response = requests.get(url, headers=headers)
# print(response.text)

items = response.json()['items']
items_list=[]
for item in items:
    title=item['title']
    link=item['link']
    thumbnail=item['thumbnail']
    sizeheight=item['sizeheight']
    sizewidth=item['sizewidth']
    
    items_list.append([title,link,thumbnail,sizeheight,sizewidth])

df=pd.DataFrame(items_list,columns=['제목','링크','썸네일','sizeheight','sizewidth'])
df.head()
df.to_csv('data/ch14_웹데이터수집1quiz(청바지).csv', index=False)

In [70]:
def save_image(attr, idx, link, query):
    '링크의 이미지를 image/attr_idx_quert.확장자로 local에 저장'
    file_extenstion = link.split('.')[-1] # 확장자
    quote_index = file_extenstion.find('?') # 확장자 뒤에 ?가 있는 위치
    if quote_index != -1:
        file_extenstion=file_extenstion[:quote_index]
    img = requests.get(link).content # 바이너리
    with open(f'image/{attr}_{idx+1}_{query}.{file_extenstion}','wb') as f:
        f.write(img)
save_image('메인',0,df.loc[0,'링크'],'청바지')

In [78]:
def get_image_list_save_image(query):
    from urllib.request import urlopen,Request
    from urllib.parse import quote
    import pandas as pd
    import os
    from dotenv import load_dotenv

    client_id = os.getenv('CLIENT_ID')
    client_secret = os.getenv('CLIENT_SECRET')

    query =quote('청바지')
    non_quote="청바지"
    headers = {
        'X-Naver-Client-Id':client_id,
        'X-Naver-Client-Secret':client_secret
    }

    url=f'https://openapi.naver.com/v1/search/image?query={query}&display=100'
    response = requests.get(url, headers=headers)

    # print(response.text)

    items = response.json()['items']
    items_list=[] # 정보가 담길 리스트
    for idx, item in enumerate(items):
        title=item['title']
        link=item['link']
        thumbnail=item['thumbnail']
        sizeheight=item['sizeheight']
        sizewidth=item['sizewidth']
        # 이미지 파일 저장
        save_image('메인',idx, link, non_quote)
        save_image('썸네일',idx, thumbnail, non_quote)

        items_list.append([title,link,thumbnail,sizeheight,sizewidth])
        
        if (idx%20==0) & (idx!=0):
            print(f'= = =  {idx}% 진행완료 = = =')
    print('= = = 완료 = = =')
    return pd.DataFrame(items_list)
df=get_image_list_save_image('청바지')
df.to_csv('image/ch14_청바지.csv')
df.sample()

= = =  20% 진행완료 = = =
= = =  40% 진행완료 = = =
= = =  60% 진행완료 = = =
= = =  80% 진행완료 = = =
= = = 완료 = = =


,0,1,2,3,4
46,여자 와이드데님 연청 청바지 : 마지아슈,http://shop1.phinf.naver.net/20250321_271/1742...,https://search.pstatic.net/common/?type=b150&s...,1000,1000


## 2.3 XML 웹데이터 수집
- RSS/ open API 을 통한 XML 웹데이터 수집
### 1) 전국 날씨 RSS를 BeautyfalSoup을 이용한 크롤링
- 구글에 "기상청 RSS" 검색한 첫번째 사이트 이동

In [93]:
import requests
from urllib.request import urlopen
from bs4 import BeautifulSoup
import numpy as np
import pandas as pd
items_list = []
url="http://www.kma.go.kr/repositary/xml/fct/mon/img/fct_mon1rss_108_20251106.xml"
# 방법1
# target = requests.get(url)
# soup=BeautifulSoup(target.text,'xml') # pip install lxml필요 없음

# 방법2
target = urlopen(url)
soup = BeautifulSoup(target,'xml')
local_tas = soup.select('local_ta')
for local_ta in local_tas:
    local_name=local_ta.select_one('local_ta_name').text
    week1_normalYear=local_ta.select_one('week1_local_ta_normalYear').text
    week1_similarRange = local_ta.select_one('week1_local_ta_similarRange').text
    week1_minVal = local_ta.select_one('week1_local_ta_minVal').text
    week1_similarVal = local_ta.select_one('week1_local_ta_similarVal').text
    week1_maxVal = local_ta.select_one('week1_local_ta_maxVal').text
    
    items_list.append({
        '지역': local_name,
        '1주평년기온': week1_normalYear,
        '1주기온범위' : week1_similarRange,
        '1주낮을확률' : week1_minVal,
        '1주비슷할확률' : week1_similarVal,
        '1주높을확률' : week1_maxVal
    })
df = pd.DataFrame(items_list)
df['1주평년기온'] = df['1주평년기온'].astype(np.float64)
df['1주낮을확률'] = df['1주낮을확률'].astype(np.int16)
df['1주비슷할확률'] = df['1주비슷할확률'].astype(np.int16)
df['1주높을확률'] = df['1주높을확률'].astype(np.int16)
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 13 entries, 0 to 12
Data columns (total 6 columns):
 #   Column   Non-Null Count  Dtype  
---  ------   --------------  -----  
 0   지역       13 non-null     object 
 1   1주평년기온   13 non-null     float64
 2   1주기온범위   13 non-null     object 
 3   1주낮을확률   13 non-null     int16  
 4   1주비슷할확률  13 non-null     int16  
 5   1주높을확률   13 non-null     int16  
dtypes: float64(1), int16(3), object(2)
memory usage: 518.0+ bytes


### 2) XML 응답하는 openAPI 활용
- data.go.kr에서
    - 서울특별시_노선정보조회 서비스(버스ID, 정류장목록과정류장ID)
    - 서울특별시_버스위치정보조회 서비스(실시간 버스 위치 목록)

In [ ]:
# step 1 버스번호의 버스id받아오기
# 서울특별시_노선정보조회 서비스 - 3번 기능(getBusRouteList) 이용

In [2]:
from dotenv import load_dotenv
import os
load_dotenv()
# print(os.getenv('KEY'))

True

In [3]:
from bs4 import BeautifulSoup
# urlretieve(url, 저장경로) : url의 파일을 저장경로에 저장
from urllib.request import urlretrieve
from urllib.parse import quote
# busNum = quote('마포01')
busNum = '152'
key = os.getenv('KEY')
url = f'http://ws.bus.go.kr/api/rest/busRouteInfo/getBusRouteList?ServiceKey={key}&strSrch={busNum}'
print(url)
savefilename1 = 'data/ch14_busInfo.xml'
urlretrieve(url, savefilename1) # xml파일(url)을 savefilename1으로 저장
with open(savefilename1, encoding='utf-8') as f:
    xml=f.read();
soup=BeautifulSoup(xml,'xml')

http://ws.bus.go.kr/api/rest/busRouteInfo/getBusRouteList?ServiceKey=d0aea5c9cf7d8e653f3f0edc0f31ab09325738011a2e39c40225f3fada1b41d0&strSrch=152


In [53]:
import requests
busNum = '162'
key = os.getenv('KEY')
url1 = f'http://ws.bus.go.kr/api/rest/busRouteInfo/getBusRouteList?ServiceKey={key}&strSrch={busNum}'
response = requests.get(url1)
soup=BeautifulSoup(response.text,'xml')

for item in soup.select('itemList'):
    busRouteNm = item.select_one('busRouteNm').text
    if busRouteNm == busNum:
        busRouteId = item.select_one('busRouteId').text
        break;
print('busRouteId =', busRouteId)

busRouteId = 100100034


In [54]:
# step2. 버스id로 버스정류장목록받아오기(정류장명, id, 경도, 위도)
# 서울특별시_노선정보조회(getStaionsByRouteList)

In [55]:
import pandas as pd
url2 =f'http://ws.bus.go.kr/api/rest/busRouteInfo/getStaionByRoute?ServiceKey={key}&busRouteId={busRouteId}'
response = requests.get(url2)
soup=BeautifulSoup(response.content,'xml')
itemLists = soup.find_all('itemList')
print(f'{busNum}번 정류장 갯수 : {len(itemLists)}')
bus_station = []
for itemList in itemLists:
    stationNm = itemList.find('stationNm').text # 정류소이름
    station = itemList.find('station').text # 정류소id
    gpsX = itemList.find('gpsX').text # 경도
    gpsY = itemList.find('gpsY').text # 위도
    
    bus_station.append([stationNm, station, gpsX, gpsY])
    
df_station = pd.DataFrame(bus_station,columns=["정류소명","id","경도","위도"])
df_station

162번 정류장 갯수 : 77


,정류소명,id,경도,위도
0,정릉산장아파트,107000071,127.003343,37.616712
1,정릉4동주민센터.경국사,107000073,127.006345,37.613529
2,북한산보국문역2번출구,107000518,127.0079858233,37.612293899
3,성북청수도서관.정릉4동성당,107000075,127.0084193769,37.6115696748
4,정릉시장입구,107000077,127.0098212542,37.6084653256
...,...,...,...,...
72,성북청수도서관.정릉4동성당,107000076,127.009045,37.610876
73,북한산보국문역1번출구,107000519,127.008329146,37.6120835499
74,정릉4동주민센터.경국사,107000074,127.006681,37.613335
75,정릉대우아파트,107000072,127.00386,37.616708


In [56]:
#step3. 버스id로 실시간위치정보 받아오기(차량번호, 혼잡도, 최종정류장id, 다음정류장id, 도착소요시간)
# 서울특별시_버스위치정보조회_서비스 - 2번(getBusPosByRtidList)

In [72]:
url3=f'http://ws.bus.go.kr/api/rest/buspos/getBusPosByRtid?serviceKey={key}&busRouteId={busRouteId}'
# print(url3)
response = requests.get(url3)
soup=BeautifulSoup(response.text,'xml')
itemLists = soup.select('itemList')
print('운행중인 버스개수 =',len(itemLists))
bus_position=[]
for itemList in itemLists:
    plainNo = itemList.select_one('plainNo').text # 차량번호
    # 혼잡도(0 : 없음, 3 : 여유, 4 : 보통, 5 : 혼잡, 6 : 매우혼잡)
    congetion = itemList.select_one('congetion').text
    congetion = "없음" if congetion=='0' \
            else '여유' if congetion == "3"\
            else '보통' if congetion=='4' else '혼잡' if congetion=='5' else '매우혼잡'
    gpsX = float(itemList.select_one('gpsX').text) # 경도
    gpsY = float(itemList.select_one('gpsY').text) # 경도
    lastStnId = itemList.select_one('lastStnId').text # 이전정류소id
    nextStId = itemList.select_one('nextStId').text # 다음정류소id
    nextStTm = int(itemList.select_one('nextStTm').text) # 다음정류소까지 소요시간
    
    bus_position.append({
        '차량번호': plainNo,
        '혼잡도' : congetion,
        '경도' : gpsX,
        '위도' : gpsY,
        "이전정류소id" : lastStnId,
        "다음정류소id" : nextStId,
        '도착소요시간' : nextStTm
    })
df_position = pd.DataFrame(bus_position)
df_position

운행중인 버스개수 = 21


,차량번호,혼잡도,경도,위도,이전정류소id,다음정류소id,도착소요시간
0,서울74사2216,여유,127.011566,37.604938,107000079,107000168,160
1,서울74사2218,보통,127.016189,37.593813,107000174,101000042,1438
2,서울70사6558,여유,126.988606,37.577426,100000076,101000042,654
3,서울74사1638,여유,126.982892,37.572642,100000106,101000042,316
4,서울74사1536,여유,126.978555,37.561273,101000032,102000018,691
5,서울74사2244,보통,126.971696,37.549572,102000012,102000018,211
6,서울74사3381,여유,126.938744,37.523502,102000052,118000061,221
7,서울74사1967,여유,126.919175,37.526780,118000071,118000070,887
8,서울74사2201,여유,126.923127,37.522780,118000071,118000070,889
9,서울70사6556,여유,126.924920,37.520614,118000070,102000053,671


In [73]:
df_position.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 21 entries, 0 to 20
Data columns (total 7 columns):
 #   Column   Non-Null Count  Dtype  
---  ------   --------------  -----  
 0   차량번호     21 non-null     object 
 1   혼잡도      21 non-null     object 
 2   경도       21 non-null     float64
 3   위도       21 non-null     float64
 4   이전정류소id  21 non-null     object 
 5   다음정류소id  21 non-null     object 
 6   도착소요시간   21 non-null     int64  
dtypes: float64(2), int64(1), object(4)
memory usage: 1.3+ KB


In [74]:
df_station.loc[df_station['id']=='107000518', '정류소명'].iloc[0]

'북한산보국문역2번출구'

In [75]:
df_station.loc[df_station['id']=='107000079', '정류소명'].iloc[0]

'정릉우체국앞'

In [76]:
def station_name(row):
#     row=row.copy() # 깊은복사
#     pd.options.mode.copy_on_write = True
    'row의 최종정류소명과 다음정류소명을 추가하고, 도착소요시간을 분으로 바꿔 return'
    row['이전정류소명'] = df_station.loc[df_station['id']==row['이전정류소id'], '정류소명'].iloc[0]
    row['다음정류소명'] = df_station.loc[df_station['id']==row['다음정류소id'], '정류소명'].iloc[0]
    m = int(row['도착소요시간']/60)
    s = row['도착소요시간']%60
    row['도착소요시간'] = f'{m}분 {s}초'
    return row
station_name(df_position.loc[21])

KeyError: 21

In [77]:
df=df_position.apply(station_name, axis = 1)

In [78]:
df

,차량번호,혼잡도,경도,위도,이전정류소id,다음정류소id,도착소요시간,이전정류소명,다음정류소명
0,서울74사2216,여유,127.011566,37.604938,107000079,107000168,2분 40초,정릉우체국앞,정릉입구.정릉역
1,서울74사2218,보통,127.016189,37.593813,107000174,101000042,23분 58초,성신여대입구역5번출구,해운센터.롯데영플라자
2,서울70사6558,여유,126.988606,37.577426,100000076,101000042,10분 54초,창덕궁.우리소리박물관,해운센터.롯데영플라자
3,서울74사1638,여유,126.982892,37.572642,100000106,101000042,5분 16초,조계사.종로경찰서,해운센터.롯데영플라자
4,서울74사1536,여유,126.978555,37.561273,101000032,102000018,11분 31초,북창동.남대문시장,남영역.민주화운동기념관
5,서울74사2244,보통,126.971696,37.549572,102000012,102000018,3분 31초,갈월동,남영역.민주화운동기념관
6,서울74사3381,여유,126.938744,37.523502,102000052,118000061,3분 41초,원효로3가,시범아파트.대교아파트
7,서울74사1967,여유,126.919175,37.526780,118000071,118000070,14분 47초,여의도역5번출구,여의도역6번출구
8,서울74사2201,여유,126.923127,37.522780,118000071,118000070,14분 49초,여의도역5번출구,여의도역6번출구
9,서울70사6556,여유,126.924920,37.520614,118000070,102000053,11분 11초,여의도역6번출구,원효로3가


In [79]:
df.drop(['이전정류소id','다음정류소id'], axis=1).head()

,차량번호,혼잡도,경도,위도,도착소요시간,이전정류소명,다음정류소명
0,서울74사2216,여유,127.011566,37.604938,2분 40초,정릉우체국앞,정릉입구.정릉역
1,서울74사2218,보통,127.016189,37.593813,23분 58초,성신여대입구역5번출구,해운센터.롯데영플라자
2,서울70사6558,여유,126.988606,37.577426,10분 54초,창덕궁.우리소리박물관,해운센터.롯데영플라자
3,서울74사1638,여유,126.982892,37.572642,5분 16초,조계사.종로경찰서,해운센터.롯데영플라자
4,서울74사1536,여유,126.978555,37.561273,11분 31초,북창동.남대문시장,남영역.민주화운동기념관


# quiz1
- yes24의 베스트셀러 정로를 제공하는 사이트에서 베스트셀러 정보를 수집해서 파일에 저장(csv또는 txt)
    - 베스트셀러 정보 수집 주소 : https://www.yes24.com/product/category/bestseller
    
```   
    - ch14_yes24 의  내용
    - 순위,책이름,저자및출판사,가격
     1,트랜드코리아2026,"김난도,전미영,최지혜,권정윤,한다혜 | 미래의창",18,000원
```

In [124]:
import requests
from bs4 import BeautifulSoup
import pandas as pd
# url = "https://www.yes24.com/product/category/bestseller?categoryNumber=001&pageNumber=1&pageSize=24"
pages=2
# response = requests.get(url)
# response.status_code
# response.text
# soup=BeautifulSoup(response.text,'html.parser')

In [155]:
books_find_list=[]
for page in range(1,pages+1):
    url = f"https://www.yes24.com/product/category/bestseller?categoryNumber=001&pageNumber={page}&pageSize=24"
    response = requests.get(url)
    soup=BeautifulSoup(response.text,'html.parser')
    
    booksRank=soup.select('em.ico.rank')
    booksType=soup.select('span.gd_res')
    booksName= soup.select('a.gd_name')
    booksWriter=soup.select('span.authPub.info_auth')
    booksPub=soup.select('span.authPub.info_pub > a')
    booksPrice=soup.select('strong.txt_num > em.yes_b')
    
    
    for rank, btype, name, writer, pub, price in zip(booksRank, booksType, booksName, booksWriter, booksPub, booksPrice):
        writer = writer.text[:40].replace('\n','').replace('\r','')
        
        # 저자가 많은 경우 스플릿을 이용해서 필요한 부분만 가져올 수 있다.
#         authors = [authors_el.text.split('정보 더 보기/감추기')[0].strip() 
#                                    for authors_el in authors_els]
        
#         print(writer)
    #     print(f'{rank.text}위 {btype.text}{name.text}, 저자|출판사: {writer} | {pub.text}, 가격 {price}')
        books_find_list.append({
            "순위" : f'{rank.text}위',
            '[분류]책이름' : f'{btype.text}{name.text}',
            '저자' : writer,
            '출판사' : pub.text,
            '가격' : f'{price.text}원'
        })
df=pd.DataFrame(books_find_list)
df.to_csv('data/ch14_yes24.csv')